In [113]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [114]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


# Parsing HTML file and get relevant information from job description

In [121]:
# # A class to represent a Webpage
import requests
from bs4 import BeautifulSoup

# Some websites need you to use proper headers when fetching them:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped.
    It stores:
      - The raw HTML (self.html)
      - The webpage title (self.title)
      - The plain text content (self.text) with script/style/img/input tags removed
      - The list of all links (self.links)
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        
        # Store the raw HTML in a string attribute
        self.html = response.text
        
        # Also store the byte content if you want both
        self.body = response.content
        
        soup = BeautifulSoup(self.html, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

    def get_html(self):
        """
        Return the raw HTML content of the webpage.
        """
        return self.html


In [120]:
jobSample = Website("https://careers.liebherr.com/job/Newport-News%2C-VA-Machine-Learning-Engineer-VA-23601/1174568301/?feedId=335501")
print(jobSample.links) # all links on the webpage
print(jobSample.get_contents())

In [117]:
system_prompt = """You are a helpful assistant. I have pasted below the raw HTML from a webpage that includes a job posting. Please thoroughly analyze the HTML and extract any and all relevant information about:

1. The Job
   - Job Title
   - Overview / Summary
   - Responsibilities / Duties
   - Requirements / Qualifications
   - Skills Needed
   - Salary / Compensation / Benefits
   - Location
   - Employment Type (full-time, part-time, contract, etc.)
   - How to Apply
   - Any other job-related information you find
   - Education Level

2. The Company
   - Company Name
   - About the Company / Description / Mission
   - Company Location(s)
   - Industry / Sector
   - products
   - Any additional context about the company
3- apply link
If the job details or company info appear in multiple sections (such as headings like "Responsibilities," "Requirements," "About Us," or in JSON-LD data), please gather them into one structured summary. Remove any duplicated or boilerplate text so the final result is clean. If something is missing, note that it's not found in the HTML.


"""

system_prompt += "You should respond in JSON as mentioned below:"


system_prompt += """
two examples are: 
{
  "job": [
    {
      "job_Title": "Software Engineer",
      "job_page": "https://example.com/jobs/software-engineer",
      "posted_date": "2025-03-08",
      "deadline": "IDK",
      "apply_link": "https://example.com/jobs/software-engineer/apply",
      "overview": "We are looking for a skilled software engineer to develop and maintain cutting-edge applications.",
      "responsibilities": "Design scalable systems, collaborate with cross-functional teams, and implement new features based on customer feedback.",
      "skills Needed": "Proficiency in Python, familiarity with cloud platforms (AWS or GCP), solid understanding of CI/CD practices.",
      "Salay": "$100,000 - $120,000 per year, plus benefits",
      "location": "New York, NY (Hybrid)",
      "Employment_type": "Full-Time",
      "how_Apply": "Complete the application form at the provided link or email your CV to hr@example.com.",
      "Others": "You must be authorized to work in the US; relocation assistance is available.",
      "Education": "Bachelor's degree in Computer Science or higher",
      "company_name": "ExampleTech Inc.",
      "About_company": "ExampleTech Inc. is a growing technology firm specializing in AI-driven solutions and modern software products.",
      "company_locations": "Headquartered in New York with additional offices in San Francisco and London",
      "Industry_sector": "Technology",
      "known_for": "ExampleTech is known for its innovative AI research and developer-friendly culture, emphasizing continuous learning and cutting-edge projects.",
      "additional_context": "Applicants should be comfortable with agile development methodologies and frequent code reviews."
    },
    {
      "type": "careers page",
      "url": "https://example.com/careers"
    }
  ]
}

example2:
{
  "job": [
    {
      "job_Title": "Digital Marketing Manager",
      "job_page": "https://anotherexample.org/careers/digital-marketing-manager",
      "posted_date": "IDK",
      "deadline": "2025-10-01",
      "apply_link": "https://anotherexample.org/careers/apply/digital-marketing-manager",
      "overview": "Seeking an experienced Digital Marketing Manager to drive online campaigns and brand awareness.",
      "responsibilities": "Develop multi-channel marketing strategies, analyze campaign performance, manage social media presence, and coordinate with design teams.",
      "skills Needed": "SEO, SEM, data analytics, email marketing tools, content strategy, and strong communication skills.",
      "Salay": "$70,000 - $90,000 per year plus performance bonuses",
      "location": "Remote (US-based)",
      "Employment_type": "Full-Time",
      "how_Apply": "Submit your resume through the application link or send an email to marketing-hr@anotherexample.org.",
      "Others": "International applicants may be considered depending on location and visa requirements.",
      "Education": "Bachelor's degree in Marketing or higher",
      "company_name": "Creative Spark Agency",
      "About_company": "A boutique creative agency offering digital marketing and brand consulting services worldwide.",
      "company_locations": "Main office in Los Angeles, additional teams in Europe and Asia",
      "Industry_sector": "Marketing and Advertising",
      "known_for": "Creative Spark is recognized for award-winning digital campaigns and innovative storytelling that resonates with global audiences.",
      "additional_context": "Candidates with strong video editing or graphic design experience are highly encouraged to apply."
    },
    {
      "type": "careers page",
      "url": "https://anotherexample.org/careers"
    }
  ]
}

"""

#*

In [118]:
def get_user_prompt(HTML, link_page):
    usr_prompt = "You are looking at an online job posting. \n"
    usr_prompt += "Here are the contents of its landing page in HTML format including  all informtion about the job and some about the company. use this information to extract job and company information.\n"
    usr_prompt += "The page link that shows the job posting is: "+ link_page+"\n"
    usr_prompt += f"TML code to parse, and find job and company related content:\n"
    usr_prompt += HTML
    
        # {HTML}
    usr_prompt += """    
        When you respond, provide:
        - If some information can not be found in the HTML code, just use IDK
        - A clearly Job Information that provides values for all the fields in the following format:
        {
        "job": [
            {"job_Title": "Title of job in page", 
            "job_page": "https://full.url/goes/here/about",
            "posted_date":"When job is posted if it is available in html code. Otherwise set the value of this field/attribute to IDK",
            "deadline":"When is the deadline if available in the html code. Otherwise set the value of this field/attribute to IDK",
            "apply_link": "decide which of the links in html code are relevant web links to apply for job",
            "overview": "summary of the job",
            "responsibilities": " list of tasks or duties for the role in job description",
            "skills Needed": " all the skills which are needed for the position",
            "Salay": "the payment for the position that is called salary, payment, compensation and benefits",
            "location": "where the job will be, remote or in-persion. Which state is the applicant will work?",
            "Employment_type": "Employment is full-time, part-time, contract, internship, etc",
            "how_Apply":"How to apply to the job. Is it through a link to external website or there is options to fill for applying on the website",
            "Others":"other job-related information that can be use to know better."
            "Education":"Does the job needs bachelor (BSc, BA), Master degree (MSc), Doctorate (PhD). If it say higher use the mentioned education and higher degree acceptable"
            "company_name":"What is the name of company",
            "About_company":"Provide information about company based on your knowledge. If you do not know about it, just put IDK for this field",
            "company_locations":"Check if the job position is in the same location as company or they are different. If the company has multiNational or national company with different locations, provide the company location and mention other locations if you have information about it",
            "Industry_sector":"provide the job is about which industry or expertise domain",
            "known_for": "provide information about what is the company known about, explain as much as you can in a paragraph",
            "additional_context":"provide information that helps job applicant to know about it and they are not mentioned in job descriptions or other attributes here"
            
            },
        ]
    }"""
    return usr_prompt

In [119]:
# And now it's time to put all of this into a function which is going to call OpenAI.
def main(url, system_prompt):
    website = Website(url)
    # completion: your task is completing this conversation
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_user_prompt(website.get_contents(),url)}
      ],
        response_format={"type": "json_object"} # we tell OpenAI, we want Json object back in its response. OpenAI in its documentation recommend that it's still important that you mention in your prompt that a json response is required even if you specify format in this argument.
    )
    result = response.choices[0].message.content 
    # dot choices zero. So what's this about? Well, as it happens we can actually in the API request ask to have multiple variations if we want, 
    # if we wanted it to generate several possible variations of the response. And we haven't done that. So we're only going to get back one.
    # Uh, and so those variations come back in the form of these choices. But we've only got one. So choices zero is getting us the one and the only choice of the response back.
    return json.loads(result)  # we use json.loads to bring it back as JSON.

In [38]:
# jobSample = Website("https://careers.liebherr.com/job/Newport-News%2C-VA-Machine-Learning-Engineer-VA-23601/1174568301/?feedId=335501")
main("https://careers.liebherr.com/job/Newport-News%2C-VA-Machine-Learning-Engineer-VA-23601/1174568301/?feedId=335501", system_prompt)

{'job': [{'job_Title': 'Machine Learning Engineer',
   'job_page': 'https://careers.liebherr.com/job/Newport-News%2C-VA-Machine-Learning-Engineer-VA-23601/1174568301/?feedId=335501',
   'posted_date': 'IDK',
   'deadline': 'IDK',
   'apply_link': 'https://careers.liebherr.com/job/Newport-News%2C-VA-Machine-Learning-Engineer-VA-23601/1174568301/?feedId=335501',
   'overview': 'The Machine Learning Engineer develops algorithms used to analyze and predict system/component health, machine performance, and application metrics for large mining haul trucks.',
   'responsibilities': 'Develops real time and post processing algorithms used to analyze machine data for failure prediction of electrical and mechanical equipment; develops algorithms used to analyze machine performance, efficiency, and application metrics; creates documentation to describe algorithm design; interfaces with other teams to create specifications; evaluates alternative Machine Learning tools and models; works independentl

In [112]:
import csv
csvfile='job_search_results.csv'
reader = csv.DictReader(csvfile)  # Use DictReader for named columns
for row in reader:
    url = row.get('link')
    if url:
        job_details = main(url, system_prompt)

print(url)

None


# Google Search based on resume

In [47]:
import fitz  # PyMuPDF

In [74]:
def get_searchTermResume_user_prompt(resume):
#     searchTermResume_user_prompt="""I have a resume and need relevant job search terms. Extract job titles that match my education, skills, experience, and projects. Ensure that:

# You consider all aspects of my resume, including past roles, technical skills, and industry expertise.
# You provide both direct job title matches and alternative roles I may qualify for.
# The job titles are commonly used in the job market and include variations.
# Organize the results into three categories:

# Primary job titles (Best matches based on my experience and skills)
# Alternative job titles (Related roles I may qualify for)
# Emerging roles (Newer job trends that align with my skill set)
# Here is my resume:\n"""
    searchTermResume_user_prompt = """I have a resume, and I need highly relevant job search terms that align with my education, skills, experience, and projects.

Extract job titles that directly match my expertise without including irrelevant suggestions.
Ensure the titles cover general job search terms that recruiters would commonly use.
Organize them into:
Primary job titles (Most relevant and directly applicable)
Alternative job titles (Closely related roles I may qualify for)
Emerging job titles (Trending roles based on my expertise)
Here is my resume:"""
    searchTermResume_user_prompt+=resume
    searchTermResume_user_prompt += "provide the search terms as a json format"
    return searchTermResume_user_prompt

In [75]:
def extract_text_from_resume(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

pdf_file = "/home/mory/Downloads/Mory_Gharasuie_resume25.pdf"
resume = extract_text_pymupdf(pdf_file)


In [76]:
# searchTermResume_system_prompt = "You are an AI assistant specialized in job searching and resume analysis. Your task is to extract relevant information from a given resume and generate effective job search queries. Focus on identifying education requirement, key job titles, skills, industries, and qualifications. Ensure that the search queries cover different variations, including synonyms, industry-specific terms, and commonly used job titles. Provide structured results categorized by job title-based searches, skill-based searches, and industry-based searches"
# searchTermResume_system_prompt = """You are an AI assistant that specializes in job search optimization. Given a resume, your goal is to extract relevant job titles that align with the candidate’s education, skills, experience, and projects. You must ensure that the extracted job titles:

# Cover a broad range of applicable roles, including direct matches and alternative job titles.
# Reflect the candidate's qualifications, industry, and expertise.
# Include commonly used variations and synonyms to maximize job search effectiveness.
# Prioritize high-relevance job roles, ensuring they match the candidate’s background.
# Your response should provide a structured list of job titles categorized by:

# Primary job titles (direct matches based on experience and skills)
# Alternative job titles (closely related roles the candidate may qualify for)
# Emerging roles (roles that fit the candidate’s skillset but may not be directly stated in experience)"""
searchTermResume_system_prompt  = """You are an AI assistant specialized in job searching and resume analysis. Your task is to extract relevant job search terms based on a candidate’s resume by identifying key job titles that align with their education, skills, experience, and projects.

To ensure high accuracy, follow these steps:

Extract job-related information: Identify the core fields of expertise from the resume, including education, skills, experience, and research areas.
Generate relevant job titles: Match job roles that directly align with the candidate’s qualifications and group them into three categories:
Primary job titles: Most relevant and high-confidence matches based on skills and experience.
Alternative job titles: Related roles that the candidate is also well-suited for.
Emerging roles: New or trending positions where the candidate’s skillset may be applicable.
Ensure generalization and relevance: Avoid overly specific or niche roles unless strongly supported by the resume. Generate job titles based on commonly used industry terms.
Examples:
Example 1
Resume Summary:

PhD in AI & Machine Learning
Experience in Deep Learning, NLP, and Computer Vision
Projects with LLMs, Semi-Supervised Learning, and Data Science
Generated Job Titles:

Primary: Machine Learning Scientist, AI Researcher, Deep Learning Engineer
Alternative: NLP Engineer, Computer Vision Engineer, Data Scientist
Emerging: LLM Engineer, AI Product Researcher, Generative AI Engineer
Example 2
Resume Summary:

Master’s in Data Science
Experience with ML, Data Engineering, and Big Data
Projects in predictive modeling and data pipelines
Generated Job Titles:

Primary: Data Scientist, ML Engineer, Data Engineer
Alternative: AI Engineer, Applied Scientist, Research Scientist
Emerging: MLOps Engineer, AI/ML Product Engineer
Use this approach to process the following resume and generate only highly relevant job search terms."""
searchTermResume_system_prompt += "You should provide very general search terms that does not have focus on specific skills. It should cover a category of jobs"

In [77]:
# And now it's time to put all of this into a function which is going to call OpenAI.
def main2(resume, system_prompt):
    # website = Website(url)
    # completion: your task is completing this conversation
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": searchTermResume_system_prompt},
            {"role": "user", "content": get_searchTermResume_user_prompt(resume)}
      ],
        response_format={"type": "json_object"} # we tell OpenAI, we want Json object back in its response. OpenAI in its documentation recommend that it's still important that you mention in your prompt that a json response is required even if you specify format in this argument.
    )
    result = response.choices[0].message.content 
    # dot choices zero. So what's this about? Well, as it happens we can actually in the API request ask to have multiple variations if we want, 
    # if we wanted it to generate several possible variations of the response. And we haven't done that. So we're only going to get back one.
    # Uh, and so those variations come back in the form of these choices. But we've only got one. So choices zero is getting us the one and the only choice of the response back.
    return json.loads(result)  # we use json.loads to bring it back as JSON.

In [79]:
job_titles = main2(resume, searchTermResume_system_prompt)

In [84]:
# !pip install serpapi
!pip show google-search-results


In [107]:
import time
import pandas as pd
from googlesearch import search
import requests

# 🔹 Define job titles based on categories

# 🔹 Add education-related search terms
education_terms = ["PhD in Computer Science"]#, "Master's in AI", "Bachelor's in Software Engineering"]

# 🔹 Combine all job titles into one search query
# all_titles = job_titles["Primary job titles"] + job_titles["Alternative job titles"] + job_titles["Emerging job titles"]
# query = " OR ".join([f'"{title}"' for title in all_titles]) + " job openings"

# # 🔹 Include education in the search query
# query += " AND (" + " OR ".join([f'"{edu}"' for edu in education_terms]) + ")"
# 🔹 Combine all job titles into one search query
all_titles = job_titles["Primary job titles"] + job_titles["Alternative job titles"] + job_titles["Emerging job titles"]
query = " OR ".join([f'"{title}"' for title in all_titles]) + " job OR career OR hiring OR opening OR 'job opening' OR 'career opportunities'"

# 🔹 Include education in the search query
query += " AND (" + " OR ".join([f'"{edu}"' for edu in education_terms]) + ")"

# 🔹 Refine the query to specific job boards/sites (e.g., LinkedIn, Indeed)
job_sites = ["site:linkedin.com", "site:indeed.com", "site:glassdoor.com", "site:google"]
query += " AND (" + " OR ".join(job_sites) + ")"

# 🔹 Define locations to filter results
locations = ["United States", "California", "New York", "Texas"]  # Modify locations as needed


# 🔹 Function to perform a job search query for each title and location

def duckduckgo_search(query):
    # DuckDuckGo search URL
    url = f"https://duckduckgo.com/html/?q={query}"
    
    # Send request to DuckDuckGo
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request fails
    
    # Parse the HTML response with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find search results (the results are in 'a' tags with class 'result__a')
    results = []
    for link in soup.find_all('a', {'class': 'result__a'}):
        title = link.get_text()
        url = link['href']
        results.append({'title': title, 'url': url})
        
    return results

# 🔹 Collect job listings for all job titles
all_job_listings = []

for category, titles in job_titles.items():
    for title in titles:
        all_job_listings.extend(search_job_titles(title))

# 🔹 Convert results to a DataFrame
df = pd.DataFrame(all_job_listings)

# 🔹 Save results to CSV
csv_filename = "job_search_results_individual_titles.csv"
df.to_csv(csv_filename, index=False)

print(f"✅ Job results saved to {csv_filename}")

# # 🔹 Fetch job listings from Google
# job_listings = []

# for location in locations:
#     # Perform search using `googlesearch` for each location
#     search_query = f"{query} {location}"
    
#     # Fetch top 10 results for the search query
#     for result in search(search_query, num_results=10):
#         job_listings.append({
#             "Title": result,  # The result title is the URL returned by search
#             "Location": location,
#             "Link": result,
#             "Snippet": "No snippet available"  # `googlesearch` doesn't provide a snippet
#         })

# # 🔹 Convert results to a DataFrame
# df = pd.DataFrame(job_listings)

# # 🔹 Save results to CSV
# csv_filename = "job_search_results_with_location.csv"
# df.to_csv(csv_filename, index=False)

# print(f"✅ Job results saved to {csv_filename}")


HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3D%2522Machine%2BLearning%2BEngineer%2522%2Bjob%2BOR%2Bcareer%2BOR%2Bhiring%2BOR%2Bopening%2BOR%2B%2522job%2Bopening%2522%2BOR%2B%2522career%2Bopportunities%2522%2BAND%2B%2528site%253Alinkedin.com%2BOR%2Bsite%253Aindeed.com%2BOR%2Bsite%253Aglassdoor.com%2BOR%2Bsite%253Agoogle%2529%2BAND%2B%2528%2522PhD%2Bin%2BComputer%2BScience%2522%2529%2BUnited%2BStates%26num%3D5%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EhAmAIgFGBRBAADEkHeIdrbLGKvzs74GIjDv6mUjC4vR126frUqbriGuDbWqJ9oLaRaLoIK5crVRCYVVOAT9R9AQpGAe8GrsZ4cyAXJaAUM

In [108]:
job_titles

{'Primary job titles': ['Machine Learning Engineer',
  'Data Scientist',
  'Software Engineer',
  'AI Researcher',
  'Computer Vision Engineer'],
 'Alternative job titles': ['Data Analyst',
  'Deep Learning Engineer',
  'NLP Engineer',
  'Full Stack Developer',
  'Software Developer'],
 'Emerging job titles': ['AI/ML Product Engineer',
  'Generative AI Engineer',
  'MLOps Engineer',
  'Chatbot Developer',
  'Artificial Intelligence Consultant']}